# Questions Clustering

## Expected
Questions to be sorted out such that the response to the whole cluster is samel.

<!--### To Do-->
Author: Sunanda Bansal  
Organization: Dataperformers  
Date: 24 Mar, 2020 (Start)  

In [5]:
import re
import os
import csv
import sys
import json
import time
import nltk
import scipy
import socket
import pickle
import numpy as np
import pandas as pd
import multiprocessing as mp

from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import Normalizer   
from sklearn import metrics   
from sklearn.metrics import confusion_matrix
# from sklearn.metrics.pairwise import paired_distances as sklearn_paired_distances

# Plotting
import seaborn as sn
import matplotlib.pyplot as plt

# importing personal development helper classes
import utils


# Disable (FALSE) displaying warnings from the OpenMP* run-time library during program execution.
os.environ['KMP_WARNINGS'] = "FALSE"

## Define variables here

Mostly the code will be intended to use with arguments that can be passed in comman line, but jupyter notebook doesn't handle `argparse` well, so the Args class is a temporary way to write the code assumming the variables to be an attribute of an object.

In [6]:
class Args:
    def __init__(self):
        # The very big scraped file, give absolute path, outside the repo
        self.dataset = "data/train_set_covid.csv"
        self.vector_mode = "tfidf"
        self.n_topics = 230
        self.dist_thresh = 1.5
        self.lang = "fr"
args=Args()

In [26]:
import re
import nltk
import unidecode

alpha_regex = re.compile('[^a-zA-Z]')

from nltk.corpus import stopwords as sw
if args.lang == "en": stopwords = sw.words('english')
if args.lang == "fr": stopwords = sw.words('french')

from nltk.stem.snowball import SnowballStemmer
if args.lang == "en": stemmer = SnowballStemmer("english")
if args.lang == "fr": stemmer = SnowballStemmer("english")

def tokenize(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            word = unidecode.unidecode(word)
            clean_words = alpha_regex.sub(' ', word).split()
            tokens.extend([word.lower() for word in clean_words])
    return tokens


def stem(word):
    return stemmer.stem(word).strip()

def preprocess(text):    
    tokenized = tokenize(text)
    cleaned = [word for word in tokenized if word not in stopwords and word is not '']
#     stemed = [stem(word) for word in cleaned]
    #stemed = [stem(word) for word in tokenized]
    #corpus[i] = ' '.join(tokenized)
    return ' '.join(tokenized)

## Dataset

In [28]:
dataset = pd.read_csv(args.dataset)

# Language Detection
def detect_lang(text):
    try:
        return detect(text)
    except:
        return "unidentifiable"   

from langdetect import detect
dataset["detected_lang"] = dataset.question.apply(detect_lang)

dataset = dataset[dataset.detected_lang == args.lang]

dataset["cleaned_question"] = dataset.question.apply(preprocess)

In [29]:
dataset.head()

,timestamp_est,anonymous_id,language,question,detected_lang,cleaned_question
6,2020-03-21 19:50:01.242,ceae6bf7-a5ec-4832-934d-211ebc182948,es,Le virus est-il aussi virulent dans les pays p...,fr,le virus est il aussi virulent dans les pays p...
16,2020-03-21 19:50:01.266,ceae6bf7-a5ec-4832-934d-211ebc182948,es,L'automédication peut-elle aider à vaincre la ...,fr,l automedication peut elle aider a vaincre la ...
30,2020-03-22 17:53:53.649,5c6d77d5-4e2a-4a3e-838d-1dce011b2631,fr,Est-ce que caresser le chien de nos amis peut ...,fr,est ce que caresser le chien de nos amis peut ...
31,2020-03-22 22:25:32.067,9f6c857a-c6da-498d-93ad-237e8310394d,fr,comment le virus se propage-t-il?,fr,comment le virus se propage t il
32,2020-03-22 01:41:10.762,3c16663e-dc8b-48f7-8168-d98020a0178e,fr,Durée du confinement,fr,duree du confinement


## LSI and AHC

In [30]:
args.n_topics = 15

In [31]:
args.dist_thresh = 0.9
model = utils.text.representation.LSI(args)
dataset["embedding"] = model.generate_embedding(dataset.question, returnarray=False)

# Cluster
X = pd.DataFrame(dataset["embedding"].values.tolist(), index= dataset.index).to_numpy()
clustering = AgglomerativeClustering(n_clusters=None, compute_full_tree=True, distance_threshold=args.dist_thresh).fit(X)
dataset["ahc_label"] = clustering.labels_

# Misc.
args.n_clusters = len(dataset["ahc_label"].unique())
print(f"Found {args.n_clusters} clusters")
dataset.groupby("ahc_label")["question"].count().sort_values()

Found 52 clusters


ahc_label
25     4
27     4
24     4
36     5
34     5
23     5
48     6
47     6
40     6
13     6
12     6
51     6
32     7
29     7
43     8
4      8
31     9
35    10
0     10
18    10
49    11
50    11
44    13
14    13
42    14
41    15
26    16
28    16
5     16
22    17
46    18
10    20
16    20
39    22
19    22
15    24
37    25
33    25
1     27
11    34
17    34
8     35
2     36
3     37
21    37
9     42
30    44
20    45
7     46
6     47
38    51
45    79
Name: question, dtype: int64

In [38]:
for label in dataset.ahc_label.unique():
    print(f"\ncluster #{label}, count - {len(dataset[dataset.ahc_label==label])}")
    print(dataset[dataset.ahc_label==label][:10].cleaned_question)


cluster #7, count - 46
6      le virus est il aussi virulent dans les pays p...
74     quelle est le coefficient de multiplication du...
91     est ce vrai que le virus reste actif dans les ...
126        le virus peut il se deposer sur des vetements
157    est ce que le virus peut etre contracte a nouv...
248    est il possible de developper des anticorps vs...
263    est ce qu on peut aller prendre une marche a l...
264    est ce que le virus peut rester sur ma carte b...
272               est ce que mon chien peut etre porteur
285    est ce que la temperature chaud ou froid peut ...
Name: cleaned_question, dtype: object

cluster #21, count - 37
16     l automedication peut elle aider a vaincre la ...
46     a partir de quelle temperature de fievre doit ...
49     quel est le de la population touchee par chaqu...
56     existe t il de nouveaux symptomes jusqu ici ig...
86             quelles sont les statistiques quebecoises
89     a partir de quelle temperature peut on conside...
2

In [12]:
dataset.groupby("ahc_label")["question"].count().describe()

count     10.000000
mean     104.100000
std      119.945775
min       16.000000
25%       42.750000
50%       68.000000
75%      112.250000
max      424.000000
Name: question, dtype: float64

In [13]:
# Generating visualization csv for clusters
counts = pd.DataFrame(list(dataset.groupby("ahc_label")["question"].apply(list))).T.count().to_frame("Count")
visual = counts.T.append(pd.DataFrame(list(dataset.groupby("ahc_label")["question"].apply(list))).T.fillna("-")).add_prefix("cluster_#")

In [14]:
visual

,cluster_#0,cluster_#1,cluster_#2,cluster_#3,cluster_#4,cluster_#5,cluster_#6,cluster_#7,cluster_#8,cluster_#9
Count,41,122,83,424,48,76,149,22,60,16
0,Quelles sont les symptômes du covid ?,J'ai une toux sèche. Devrais je faire un test ...,combien de temps le virus survit-ils sur les s...,Le virus est-il aussi virulent dans les pays p...,comment le virus se propage-t-il?,En parallèle avec le covid-19 combien de cas d...,Est-ce que je peux laisser les fenêtres ouvert...,Est-ce que les personnes asthmatiques sont plu...,Doit-on mettre notre linge au lavage à chaque ...,Ma conjointe a des symptômes. Dois-je me faire...
1,Quels sont les symptômes?,"J’ai de la toux, mais pas de fièvre. Suis-je i...",Pendant combien de temps durera l'épidémie du ...,L'automédication peut-elle aider à vaincre la ...,Comment se propage le virus?,"Il y a beaucoup d'information contradictoires,...",Si une personne tousse devant moi et que je re...,"Je suis enceinte, est ce que j’ai plus de risq...",Comment obtenir un traitement urgent d'un phys...,Comment ca marche pour se faire tester?
2,Quels sont les premiers symptomes du covid-19?,Que dois-je faire si j’ai certains symptômes ?,Combien de temps avant d’avoir les résultats d...,Est-ce que caresser le chien de nos amis peut ...,Comment se faire dépister ?,Quels sont les types de traitement en expérime...,Je tousse dpeuis hier je fais koi ?,Y a-t-il des dangers pour les femmes enceintes ?,Où sont traiter les cas de covid19 ?,Ou je dois aller pour me faire tester ?
3,Quels sont les symptômes du Covid ?,"Si je rencontre quelqu'un positif au virus, co...",Combien de temps cela prend pour avoir les rés...,Durée du confinement,Comment distinguer le virus de la grippe au Co...,Que dois-je faire si je présente des symptômes...,"Bonjour, puis je me réunir avec des amis ?",Quels sont les risques du COVID19 pour les fem...,Est-ce que la situation va empirer au Québec?,Où puis-je me faire tester ?
4,Quels sont les symptomes,J'ai besoin d'aide d'une gardienne pour s'occu...,Combien de temps est ce que le virus reste act...,Est-ce que les rendez-vous de suivi de grosses...,Comment peut-on contracter le covid19?,Est-il vrai que la Covid-19 cause des nausées ...,Que faire quand on a des doutes sur sa santé,Quel est le risque de contamination pour les f...,Combien de morts aujourd’hui dans le monde?,Comment puis-je me faire tester?
5,Quels sont les manifestations du covid 19 chez...,Si je n'ai pas de fièvre et que je ne me sens ...,combien de temps faut il avant d'avoir les sym...,Pourquoi ne pas obliger le port du masque par ...,Comment attrape t-on le Covid-19?,Quel est le taux de survie de la COVID-19?,Est-ce que je vais perdre mon emploi?,Quels sont les risques pour les femmes enceint...,Est ce que le Coronavirus part au lavage ?,A quel moment je dois me faire tester?
6,Quels sont les symptômes de la covid 19 ?,"Bonjour, j’ai appris que dans le bâtiment où m...",Combien de temps le virus demeure sur les surf...,Pourquoi ne pas tout arrêter l'économie comme ...,"Dans les batiments publics, ils sont tous muni...",est-ce qu'on peut attraper la Covid-19 plus d'...,"Si je vais nager dans une piscine, suis-je à r...",Âges à risque,Quel est le nombre de tests effectués au Québe...,Comment puis-je me faire tester?
7,Quels sont les symptômes?,Je reviens de voyage. Je suis en isolement vol...,Combien de temps le Coronavirus reste il sur l...,Va-t-on être en pénurie de nourriture?,comment désinfecter l’épicerie?,Qu'est-ce que la covid,"Je suis enrhumée, est-ce que je suis plus vuln...",Est-ce que je suis à risques pour le covid-19?,Quels sont les scénarios de cette pandémie ici...,Que dois-je faire pour me faire tester ?
8,Quelles sont les symptômes de la covid-19,Que faire si j’ai des symptômes?,Combien de temps le covid-19 survit-il?,En toute transparence. Le port du masque n'est...,comment est-ce que les garderies en milieu fam...,Est-ce qu'il existe actuellement un médicament...,Devrais-je faire obligatoirement du télé-tr

In [55]:
dataset.sort_values("ahc_label")[["question","ahc_label"]].to_csv(f"output/{args.n_topics}lsa_{args.dist_thresh}dt_{args.n_clusters}ahcx.csv")
visual.to_csv(f"output/clusters_{args.n_topics}lsa_{args.dist_thresh}dt_{args.n_clusters}ahcx.csv")

# LEVEL 2

In [228]:
dataset = pd.read_csv("output/text_rules_based_lv2.csv").fillna("unclassified")
dataset = dataset[["question", "cluster"]]
d = dataset[dataset.cluster=="unclassified"]
d.head()

,question,cluster
0,Are research fundings going to be cut as a res...,unclassified
1,Is it ok to go outside to get fresh air,unclassified
2,Should I star home and even avoid going to the...,unclassified
3,Are we going to have trouble buying milk,unclassified
4,Will Covid19 go away like the 1918 influenza a...,unclassified


In [235]:
args.n_topics = 15
args.dist_thresh = 0.6
model = utils.text.representation.LSI(args)
d["embedding"] = model.generate_embedding(d.question, returnarray=False)

# Cluster
X = pd.DataFrame(d["embedding"].values.tolist(), index= d.index).to_numpy()
clustering = AgglomerativeClustering(n_clusters=None, compute_full_tree=True, distance_threshold=args.dist_thresh).fit(X)
d["ahc_label"] = clustering.labels_

# Misc.
args.n_clusters = len(d["ahc_label"].unique())
print(f"Found {args.n_clusters} clusters")
d.groupby("ahc_label")["question"].count().sort_values()

Found 32 clusters


ahc_label
21     2
20     2
18     2
9      2
31     3
22     3
19     3
12     3
25     3
29     4
28     4
27     4
17     4
7      4
8      4
26     4
23     4
11     4
10     4
4      5
16     5
30     6
3      6
6      6
14     6
15     8
5     10
13    12
24    12
0     13
1     15
2     83
Name: question, dtype: int64

In [236]:
for label in d.ahc_label.unique():
    print(f"\ncluster #{label}, count - {len(d[d.ahc_label==label])}")
    print(d[d.ahc_label==label][:10].question)

cluster #5, count - 10
0     Are research fundings going to be cut as a res...
1               Is it ok to go outside to get fresh air
2     Should I star home and even avoid going to the...
3              Are we going to have trouble buying milk
4     Will Covid19 go away like the 1918 influenza a...
6                        when covid 19 is going to stop
7                            When should I go to the ER
9                         Am I allowed to go for a walk
10                     When will it be ok to go outside
11    how much fever should i tolerate before going ...
Name: question, dtype: object
cluster #20, count - 2
5    Where should I go
8         how is going
Name: question, dtype: object
cluster #24, count - 12
12         Is there more than one strain of COVID 19
343    What do I do if someone with COVID 19 spotted
344                         COVID 19 stands for what
345                           Why is called COVID 19
346                     Where did COVID 19 come from

In [217]:
d.head()

,question,cluster,embedding,ahc_label
0,Are research fundings going to be cut as a res...,unclassified,"[0.03784225848917501, 0.14792442906499279, -0....",11
1,Is it ok to go outside to get fresh air,unclassified,"[0.10863800543379393, 0.19925573895788562, -0....",11
2,Should I star home and even avoid going to the...,unclassified,"[0.029156588250339985, 0.039139206493714655, -...",11
3,Are we going to have trouble buying milk,unclassified,"[0.03265145602538317, 0.028593934327626164, -0...",11
4,Will Covid19 go away like the 1918 influenza a...,unclassified,"[0.019492929504519453, 0.03203605628662124, -0...",11


In [231]:
d.groupby("ahc_label")["question"].count().describe()

count    71.000000
mean      3.521127
std       6.626482
min       1.000000
25%       1.000000
50%       2.000000
75%       4.000000
max      55.000000
Name: question, dtype: float64

In [141]:
# d.sort_values("ahc_label")[["question","ahc_label"]].to_csv(f"output/lv2_{args.n_topics}lsa_{args.dist_thresh}dt_{args.n_clusters}ahcx.csv")

In [232]:
dataset = dataset.join(d["ahc_label"])

In [220]:
dataset.sort_values("ahc_label")[["question","ahc_label","cluster"]]

,question,ahc_label,cluster
133,Does Wearing gloves help,0.0,unclassified
308,Should I wear mask,0.0,unclassified
310,Why does the government think that we don t ne...,0.0,unclassified
307,Is it useful to wear a mask if we are not sick,0.0,unclassified
311,Does a mask actually help if I m not sick,0.0,unclassified
309,Do I need a mask,0.0,unclassified
93,When this virus terminated,1.0,unclassified
111,What is the difference between a bacteria and ...,1.0,unclassified
110,how can one fight the virus,1.0,unclassified
109,Can this virus infect me in an airborne fashion,1.0,unclassified


In [233]:
dataset.sort_values("ahc_label")[["question","ahc_label","cluster"]].to_csv(f"output/text_rules_based_plusplus_{args.n_topics}lsa_{args.dist_thresh}dt_{args.n_clusters}ahcx.csv")

In [177]:
dataset = pd.read_csv(f"output/text_rules_based_plus_{args.n_topics}lsa_{args.dist_thresh}dt_{args.n_clusters}ahcx_manual.csv")

In [180]:
dataset["cluster"] = dataset.rule_based
dataset["cluster"][dataset.rule_based=="unclassified"] = dataset.ahc_manual_clutser_label

/home/designer1/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [184]:
len(dataset[dataset.cluster.isna()])

250

In [185]:
# dataset.to_csv("output/text_rules_based_lv2.csv")

In [ ]:
# Generating visualization csv for clusters
counts = pd.DataFrame(list(dataset.groupby("cluster")["question"].apply(list))).T.count().to_frame("Count")
visual = counts.T.append(pd.DataFrame(list(dataset.groupby("ahc_label")["question"].apply(list))).T.fillna("-")).add_prefix("cluster_#")

In [173]:
pd.DataFrame(list(dataset.groupby(["cluster","ahc_label"])["question"].apply(list))).T.count().to_frame("Count")

,Count
0,5
1,7
2,1
3,2
4,7
5,7
6,6
7,2
8,1
9,2
